# Importing Libraries


In [257]:
import helper
from helper import *

In [258]:
import importlib
importlib.reload(helper)


<module 'helper' from 'c:\\Users\\satya\\OneDrive\\Desktop\\Summer_Internship\\helper.py'>

# Data Collection and cleaning

In [3]:
# basic variables

start_date = '2000-01-01'
end_date = '2023-12-31' 
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

In [4]:
symbols = collect_tickers('US', start_date, end_date)

Sucesfully downloaded 835 symbols.


In [5]:
snp_stocks_data = download_data(symbols, start_date, end_date)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

$BF.B: possibly delisted; No price data found  (1d 2000-01-01 -> 2023-12-31)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Could not fetch data for nan: 'float' object has no attribute 'replace'
Could not fetch data for nan: 'float' object has no attribute 'replace'


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['ATVI']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed


...

$DJ: possibly delisted; No price data found  (1d 2000-01-01 -> 2023-12-31)
$AV: possibly delisted; No price data found  (1d 2000-01-01 -> 2023-12-31)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['NCR']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['MXIM']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['FDC']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed


Could not fetch data for nan: 'float' object has no attribute 'replace'


[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['TSG']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['QTRN']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2000-01-01 -> 2023-12-31)')


$QTRN: possibly delisted; No price data found  (1d 2000-01-01 -> 2023-12-31)
Sucesfully downloaded data for 799 stocks!


In [6]:
snp_stocks_data = clean_data(symbols, snp_stocks_data, date_range)
snp_stocks_data = clean_data(symbols, snp_stocks_data, date_range)
snp_stocks_data = clean_data(symbols, snp_stocks_data, date_range)

Could not update data for 16 stocks.
Could not update data for 3 stocks.


In [7]:
symbols = list(snp_stocks_data.keys())

In [8]:
returns_df = calculate_return(symbols, snp_stocks_data, end_date)

In [9]:
momentum_data = {}
for symbol, data in snp_stocks_data.items():
    momentum_data[symbol] = calculate_m_momentum(data, 12)

# Convert the momentum data dictionary to a DataFrame
momentum = pd.DataFrame.from_dict(momentum_data, orient='index')

c:\Users\satya\OneDrive\Desktop\Summer_Internship\helper.py:140: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if monthly_data[i-12] != 0:
c:\Users\satya\OneDrive\Desktop\Summer_Internship\helper.py:141: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  momentum_return = ((monthly_data[i] - monthly_data[i-12]) / monthly_data[i-12]) * 100


In [10]:
# Calculate the 12-1 momentum for each stock in the snp_stocks_data dictionary
momentum_data = {}
for symbol, data in snp_stocks_data.items():
    momentum_data[symbol] = calculate_12_1_momentum(data)

# Convert the momentum data dictionary to a DataFrame
momentum_12_1 = pd.DataFrame.from_dict(momentum_data, orient='index')

c:\Users\satya\OneDrive\Desktop\Summer_Internship\helper.py:159: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if monthly_data[i-12] != 0:
c:\Users\satya\OneDrive\Desktop\Summer_Internship\helper.py:160: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  momentum_return = (monthly_data[i-1] - monthly_data[i-12]) / monthly_data[i-12] * 100


In [252]:
# Calculate the 3-months momentum for each stock in the snp_stocks_data dictionary
momentum_data = {}
for symbol, data in snp_stocks_data.items():
    momentum_data[symbol] = calculate_m_momentum(data, 3)

# Convert the momentum data dictionary to a DataFrame
momentum_3 = pd.DataFrame.from_dict(momentum_data, orient='index')

c:\Users\satya\OneDrive\Desktop\Summer_Internship\helper.py:140: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

c:\Users\satya\OneDrive\Desktop\Summer_Internship\helper.py:141: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [11]:
top_p_momentum_stocks_dict = top_p_percentile_stocks(momentum, 10)

In [12]:
top_30_momentum = top_p_percentile_stocks(momentum, 10)

In [95]:
"""
  Selects stocks from dict2 that are not present in dict1.

  Args:
      dict1 (dict): Dictionary with timestamps as keys and lists of timestamps as values.
      dict2 (dict): Dictionary with timestamps as keys and lists of timestamps (stock symbols) as values.

  Returns:
      list: List of stock symbols from dict2 that are not in dict1.
"""
def select_unique_stocks(dict1, dict2):
    unique_stocks_dict = {}

    for timestamp in dict2:
        # Get the list of stocks for the current timestamp from dict2
        stocks_in_dict2 = dict2[timestamp]
        
        # Get the list of stocks for the current timestamp from dict1, if it exists
        stocks_in_dict1 = dict1[timestamp]
        
        # Select stocks that are in dict2 but not in dict1
        unique_stocks = [stock for stock in stocks_in_dict2 if stock not in stocks_in_dict1]
        
        # Store the unique stocks in the result dictionary
        unique_stocks_dict[timestamp] = unique_stocks
    
    return unique_stocks_dict



In [96]:
top11_30_momentum = select_unique_stocks(top_p_momentum_stocks_dict, top_30_momentum)

In [97]:
top11_30_momentum

{Timestamp('2001-01-31 00:00:00'): ['LH',
  'DGX',
  'EOG',
  'COR',
  'EG',
  'BLK',
  'HSIC',
  'FL',
  'UNH',
  'CB',
  'WRB',
  'TDY',
  'SRCL',
  'CAH',
  'PHM',
  'WAT',
  'LUV',
  'AJG',
  'PVH',
  'PPL',
  'UHS',
  'L',
  'EQT',
  'CNP',
  'ALL',
  'HIG',
  'SLM',
  'TRV',
  'MCK',
  'ESS'],
 Timestamp('2001-02-28 00:00:00'): ['DVA',
  'DGX',
  'RRC',
  'EOG',
  'EG',
  'NI',
  'UHS',
  'DHI',
  'LEN',
  'TRV',
  'CB',
  'ALL',
  'HIG',
  'PHM',
  'AJG',
  'PXD',
  'OKE',
  'UNM',
  'HUM',
  'ESRX',
  'FL',
  'CMS',
  'PNC',
  'FLS',
  'FI',
  'AET',
  'BLK',
  'TMO',
  'GL',
  'IVZ',
  'PDCO',
  'KSS',
  'CTRA'],
 Timestamp('2001-03-31 00:00:00'): ['DVA',
  'COR',
  'LH',
  'RRC',
  'IGT',
  'DGX',
  'CAH',
  'ESRX',
  'TIE',
  'EOG',
  'THC',
  'WM',
  'BRO',
  'BLK',
  'LMT',
  'OKE',
  'NOC',
  'PDCO',
  'EVRG',
  'POOL',
  'WBA',
  'FHN',
  'CVS',
  'VRTX',
  'PNC',
  'CMA',
  'FICO'],
 Timestamp('2001-04-30 00:00:00'): ['ESRX',
  'SLM',
  'EG',
  'SRCL',
  'WRB',
  'MET',

In [15]:
top_p_momentum_stocks_dict_12_1 = top_p_percentile_stocks(momentum_12_1, 10)

In [16]:
bottom_p_momentum_stocks_dict = bottom_p_percentile_stocks(momentum, 10)

In [17]:
bottom_p_momentum_stocks_dict_12_1 = bottom_p_percentile_stocks(momentum_12_1, 10)

In [18]:
long_dict = total_return(snp_stocks_data, top_p_momentum_stocks_dict, returns_df)

In [98]:
long_dict_11_30 = total_return(snp_stocks_data, top11_30_momentum, returns_df)

In [140]:
m = 6
momentum_6_ma = stocks_moving_average(momentum_dict=top_p_momentum_stocks_dict, m= 6 ,stock_data_dict=snp_stocks_data)

In [141]:
above_ma_returns = total_return(snp_stocks_data, momentum_6_ma, returns_df)

In [186]:
long_dict_12_1 = total_return(snp_stocks_data, top_p_momentum_stocks_dict_12_1, returns_df)

In [21]:
long_dict_inversion = total_return(snp_stocks_data, bottom_p_momentum_stocks_dict, returns_df)

In [22]:
sort_dict = total_return(snp_stocks_data, bottom_p_momentum_stocks_dict, returns_df)

In [23]:
sort_dict_12_1 = total_return(snp_stocks_data, bottom_p_momentum_stocks_dict_12_1, returns_df)

In [24]:
sort_dict_inversion = total_return(snp_stocks_data, top_p_momentum_stocks_dict, returns_df)

In [80]:
sort_long_dict = long_sort_return(snp_stocks_data, top_p_momentum_stocks_dict, bottom_p_momentum_stocks_dict, returns_df)

In [28]:
sort_long_dict

{Timestamp('2001-01-31 00:00:00'): 2.9820069431841945,
 Timestamp('2001-02-28 00:00:00'): 0.21929460894389172,
 Timestamp('2001-03-31 00:00:00'): -6.797164047456317,
 Timestamp('2001-04-30 00:00:00'): -0.12200590497981557,
 Timestamp('2001-05-31 00:00:00'): 0.9149632908227946,
 Timestamp('2001-06-30 00:00:00'): 4.128943225259209,
 Timestamp('2001-07-31 00:00:00'): 9.045645274307718,
 Timestamp('2001-08-31 00:00:00'): 2.6938284190646584,
 Timestamp('2001-09-30 00:00:00'): -0.34648969040367145,
 Timestamp('2001-10-31 00:00:00'): -3.884143582481503,
 Timestamp('2001-11-30 00:00:00'): -2.984080882766058,
 Timestamp('2001-12-31 00:00:00'): 4.971197978045767,
 Timestamp('2002-01-31 00:00:00'): 0.3397027501592485,
 Timestamp('2002-02-28 00:00:00'): -6.345164129537685,
 Timestamp('2002-03-31 00:00:00'): 0.2581500254168176,
 Timestamp('2002-04-30 00:00:00'): 1.030879575704515,
 Timestamp('2002-05-31 00:00:00'): 3.7072750076317558,
 Timestamp('2002-06-30 00:00:00'): 2.1934131125378253,
 Timestam

In [30]:

# Apply the trading strategy
trade_returns_seasonality = seasonality_strategy(returns_df, top_p_momentum_stocks_dict)


In [31]:

x = returns_df.loc[['GR', 'NVR', 'ODP', 'TWX', 'BKNG', 'IBM'],'2000-06-30'].abs()
x[x<=0].empty

False

# Benchmark

The S&P500 index is used as a benchmark.

In [32]:
data = benchmark_data('US', start_date, end_date)

[*********************100%%**********************]  1 of 1 completed


In [33]:
gains = benchmark_returns(data)
gains.index = pd.to_datetime(gains.index, format='%Y-%m-%d')

In [115]:
# Define a function to calculate 12-month momentum for S&P 500
def calculate_12_month_momentum(data):
    return data.pct_change(12).dropna()

benchmark = data.loc[:'2024-05-31 00:00:00']
sp500_momentum = pd. DataFrame(calculate_12_month_momentum(benchmark))

In [122]:
momentum

,2001-01-31,2001-02-28,2001-03-31,2001-04-30,2001-05-31,2001-06-30,2001-07-31,2001-08-31,2001-09-30,2001-10-31,...,2023-03-31,2023-04-30,2023-05-31,2023-06-30,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2023-12-31
MMM,21.788978,31.567436,20.727142,41.377130,42.070564,41.232502,27.624856,14.917001,10.878404,10.921997,...,-25.485843,-22.265844,-33.633585,-17.875302,-17.346457,-8.407384,-9.537146,-22.799427,-15.626109,-2.199513
AOS,-11.659505,10.838917,12.013344,-3.951198,-17.117905,-11.624347,12.205431,10.420755,44.106860,15.261935,...,10.405842,19.220401,8.488327,35.772821,17.032403,30.936883,38.780649,29.690778,26.354503,46.677332
ABT,40.224684,51.104000,36.503689,22.736187,29.948832,9.564269,31.340240,15.500913,10.878209,2.044133,...,-12.925119,-0.837866,-11.527655,2.228924,4.249237,2.165814,2.012391,-2.525298,-1.118875,2.260618
ABBV,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.051955,6.877281,-2.754335,-8.621292,8.458169,13.730482,15.570519,0.330194,-8.088916,-0.234836
ACN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-13.963180,-5.177903,4.151549,12.931236,4.967010,14.058137,21.291247,6.308597,12.462324,33.595133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TSG,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
SBL,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ABS,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
GLK,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:

results_df = calculate_yearly_stats(sort_long_dict, gains)


c:\Users\satya\OneDrive\Desktop\Summer_Internship\helper.py:353: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.

c:\Users\satya\OneDrive\Desktop\Summer_Internship\helper.py:365: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.



# Plots

## 12-months momentum and 1-month return

### Going long only for the top 10% stocks

In [373]:
# Convert the total_returns_dict to a DataFrame
total_returns_df = pd.DataFrame.from_dict(long_dict, orient='index', columns=['Total Returns'])
total_returns_df.sort_index(inplace=True)


In [410]:
# Create a Plotly figure
fig = go.Figure()

# Add the benchmark returns bar trace
fig.add_trace(
    go.Bar(
        x=gains.index,
        y=gains.values,
        name='Benchmark Returns'
    )
)

# Add the momentum returns bar trace
fig.add_trace(
    go.Bar(
        x=total_returns_df.index,
        y=total_returns_df['Total Returns'],
        name='Momentum Returns'
    )
)

# Update the layout to include a range slider
fig.update_layout(
    title='Momentum Return vs Benchmark Return from 01-01-2000',
    xaxis_title='Date',
    yaxis_title='Return %',
    barmode='group',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

# Show the plot
fig.show()

In [411]:
# Ensure both DataFrames have the same index for correct comparison
aligned_benchmark_gains = gains.reindex(total_returns_df.index).fillna(0)

# Calculate cumulative returns
cumulative_benchmark_returns = (1 + aligned_benchmark_gains / 100).cumprod() - 1
cumulative_momentum_returns = (1 + total_returns_df['Total Returns'] / 100).cumprod() - 1


In [413]:
pd.Series(long_dict).mean()

0.4381872662838454

In [414]:
# Calculate Sharpe ratios
sharpe_ratio_benchmark, x = calculate_sharpe_ratio(gains)
sharpe_ratio_momentum, y = calculate_sharpe_ratio(pd.Series(long_dict))


# Create a Plotly figure
fig = go.Figure()

# Add the cumulative benchmark returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_benchmark_returns.index,
        y=cumulative_benchmark_returns * 100,
        mode='lines',
        name='Cumulative Benchmark Returns'
    )
)

# Add the cumulative momentum returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_momentum_returns.index,
        y=cumulative_momentum_returns * 100,
        mode='lines',
        name='Cumulative Momentum Returns (Long-Only)'
    )
)

# Configure the x-axis to show months when zoomed in and add range selector buttons
fig.update_layout(
    title='Cumulative Returns: Momentum (Long-Only) vs Benchmark from 01-01-2000',
    xaxis_title='Date',
    yaxis_title='Cumulative Return %',
    xaxis=dict(
        tickmode='auto',
        rangeslider=dict(
            visible=True
        ),
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        type="date"
    ),
)

# Add annotations for Sharpe ratios and final cumulative returns
annotation_text = (
    f"<b>Sharpe Ratio</b><br>"
    f"Benchmark: {sharpe_ratio_benchmark:.2f}<br>"
    f"Momentum: {sharpe_ratio_momentum:.2f}<br><br>"
    f"<b>Final Cumulative Return</b><br>"
    f"Benchmark: {final_cumulative_return_benchmark:.2f}%<br>"
    f"Momentum: {final_cumulative_return_momentum:.2f}%"
)

fig.add_annotation(
    xref='paper', yref='paper',
    x=1.05, y=0.5,
    xanchor='left', yanchor='middle',
    text=annotation_text,
    showarrow=False,
    font=dict(size=12)
)

# Show the plot
fig.show()

In [377]:
# Calculate the yearly stats
yearly_stats_df = calculate_yearly_stats(long_dict, gains)

yearly_stats_df.T

Year,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Strategy Returns,-0.80,32.71,11.33,18.52,0.95,0.33,-32.76,16.79,-2.20,1.91,...,14.82,-0.93,-19.43,14.17,-20.32,6.46,18.32,1.57,2.93,25.29
Benchmark Returns,-21.81,27.10,9.25,3.29,13.79,3.97,-37.08,26.30,14.71,1.15,...,11.72,0.11,10.05,19.50,-5.19,29.85,19.87,27.59,-17.44,25.42
Strategy Sharpe Ratio,-0.00,0.25,0.06,0.07,0.00,0.00,-0.10,0.08,-0.01,0.01,...,0.04,-0.01,-0.15,0.04,-0.07,0.04,0.06,0.00,0.04,0.09
Benchmark Sharpe Ratio,-0.10,0.18,0.10,0.03,0.19,0.03,-0.18,0.09,0.06,0.01,...,0.11,0.00,0.08,0.38,-0.03,0.17,0.06,0.19,-0.07,0.13


### Going long on top 10% and short on bottom 10%

In [81]:
# Convert the total_returns_dict to a DataFrame
total_returns_df = pd.DataFrame.from_dict(sort_long_dict, orient='index', columns=['Total Returns'])
total_returns_df.sort_index(inplace=True)


In [82]:
# Create a Plotly figure
fig = go.Figure()

# Add the benchmark returns bar trace
fig.add_trace(
    go.Bar(
        x=gains.index,
        y=gains.values,
        name='Benchmark Returns'
    )
)

# Add the momentum returns bar trace
fig.add_trace(
    go.Bar(
        x=total_returns_df.index,
        y=total_returns_df['Total Returns'],
        name='Momentum Returns'
    )
)

# Update the layout to include a range slider
fig.update_layout(
    title='Momentum Return(long and sort) vs Benchmark Return from 01-01-2000',
    xaxis_title='Date',
    yaxis_title='Return %',
    barmode='group',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

# Show the plot
fig.show()

In [83]:
# Ensure both DataFrames have the same index for correct comparison
aligned_benchmark_gains = gains.reindex(total_returns_df.index).fillna(0)

# Calculate cumulative returns
cumulative_benchmark_returns = (1 + aligned_benchmark_gains / 100).cumprod() - 1
cumulative_momentum_returns = (1 + total_returns_df['Total Returns'] / 100).cumprod() - 1


In [406]:

# Calculate Sharpe ratios
sharpe_ratio_benchmark, x = calculate_sharpe_ratio(gains)
sharpe_ratio_momentum, y = calculate_sharpe_ratio(pd.Series(long_dict))

# Final cumulative returns
final_cumulative_return_benchmark = cumulative_benchmark_returns.iloc[-1] * 100
final_cumulative_return_momentum = cumulative_momentum_returns.iloc[-1] * 100

# Create a Plotly figure
fig = go.Figure()

# Add the cumulative benchmark returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_benchmark_returns.index,
        y=cumulative_benchmark_returns * 100,
        mode='lines',
        name='Cumulative Benchmark Returns'
    )
)

# Add the cumulative momentum returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_momentum_returns.index,
        y=cumulative_momentum_returns * 100,
        mode='lines',
        name='Cumulative Momentum Returns (Long-Only)'
    )
)

# Configure the x-axis to show months when zoomed in and add range selector buttons
fig.update_layout(
    title='Cumulative Returns: Momentum (Long-Only) vs Benchmark from 01-01-2000',
    xaxis_title='Date',
    yaxis_title='Cumulative Return %',
    xaxis=dict(
        tickmode='auto',
        rangeslider=dict(
            visible=True
        ),
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        type="date"
    ),
)

# Add annotations for Sharpe ratios and final cumulative returns
annotation_text = (
    f"<b>Sharpe Ratio</b><br>"
    f"Benchmark: {sharpe_ratio_benchmark:.2f}<br>"
    f"Momentum: {sharpe_ratio_momentum:.2f}<br><br>"
    f"<b>Final Cumulative Return</b><br>"
    f"Benchmark: {final_cumulative_return_benchmark:.2f}%<br>"
    f"Momentum: {final_cumulative_return_momentum:.2f}%"
)

fig.add_annotation(
    xref='paper', yref='paper',
    x=1.05, y=0.5,
    xanchor='left', yanchor='middle',
    text=annotation_text,
    showarrow=False,
    font=dict(size=12)
)

# Show the plot
fig.show()

In [86]:
results_df = calculate_yearly_stats(sort_long_dict, gains)
results_df.T.round(3)

Year,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Strategy Returns,6.767,-9.303,1.339,-3.019,-4.971,8.211,15.305,-16.014,-9.853,23.382,...,0.559,14.469,1.918,-7.288,5.783,-11.888,-19.981,4.118,-5.617,1.496
Benchmark Returns,-21.809,27.100,9.254,3.287,13.787,3.975,-37.084,26.305,14.705,1.152,...,11.723,0.113,10.052,19.500,-5.195,29.852,19.868,27.586,-17.436,25.423
Strategy Sharpe Ratio,0.048,-0.108,0.044,-0.032,-0.095,0.069,0.080,-0.101,-0.124,0.126,...,0.048,0.069,0.015,-0.064,0.023,-0.071,-0.114,0.070,-0.037,0.020
Benchmark Sharpe Ratio,-0.098,0.177,0.102,0.035,0.190,0.034,-0.180,0.088,0.060,0.006,...,0.114,0.001,0.078,0.385,-0.029,0.170,0.059,0.186,-0.069,0.130


### 12-1 momentum strategy

In [359]:
# Convert the total_returns_dict to a DataFrame
total_returns_df_12_1 = pd.DataFrame.from_dict(long_dict_12_1, orient='index', columns=['Total Returns'])
total_returns_df_12_1.sort_index(inplace=True)


In [360]:
# Create a Plotly figure
fig = go.Figure()

# Add the benchmark returns bar trace
fig.add_trace(
    go.Bar(
        x=gains.index,
        y=gains.values,
        name='Benchmark Returns'
    )
)

# Add the momentum returns bar trace
fig.add_trace(
    go.Bar(
        x=total_returns_df_12_1.index,
        y=total_returns_df_12_1['Total Returns'],
        name='12-1 Momentum Returns'
    )
)

# Update the layout to include a range slider
fig.update_layout(
    title='12-1 Momentum Return vs Benchmark Return from 01-01-2000',
    xaxis_title='Date',
    yaxis_title='Return %',
    barmode='group',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

# Show the plot
fig.show()

In [361]:
# Ensure both DataFrames have the same index for correct comparison
aligned_benchmark_gains_12_1 = gains.reindex(total_returns_df_12_1.index).fillna(0)

# Calculate cumulative returns
cumulative_benchmark_returns_12_1 = (1 + aligned_benchmark_gains_12_1 / 100).cumprod() - 1
cumulative_momentum_returns_12_1 = (1 + total_returns_df_12_1['Total Returns'] / 100).cumprod() - 1


In [362]:

# Create a Plotly figure
fig = go.Figure()

# Add the cumulative benchmark returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_benchmark_returns_12_1.index,
        y=cumulative_benchmark_returns_12_1,
        mode='lines',
        name='Cumulative Benchmark Returns'
    )
)

# Add the cumulative momentum returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_momentum_returns_12_1.index,
        y=cumulative_momentum_returns_12_1,
        mode='lines',
        name='12-1 Cumulative Momentum Returns'
    )
)

# Configure the x-axis to show months when zoomed in and add range selector buttons
fig.update_layout(
    title='Cumulative Returns: 12-1 Momentum vs Benchmark from 01-01-2001',
    xaxis_title='Date',
    yaxis_title='Cumulative Return %',
    xaxis=dict(
        tickmode='auto',
        rangeslider=dict(
            visible=True
        ),
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        type="date"
    ),
)

# Show the plot
fig.show()

In [87]:
# Calculate the yearly stats
yearly_stats_df = calculate_yearly_stats(long_dict_12_1, gains)

yearly_stats_df.T

Year,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Strategy Returns,-2.618118,24.749490,12.168707,18.977095,-0.606356,-5.608893,-30.870343,1.003860,-9.320357,1.217590,...,-5.309770,-21.762780,-8.201571,10.960840,-18.761860,8.860323,3.638444,0.499638,-3.094252,13.740399
Benchmark Returns,-21.809476,27.099534,9.254411,3.286917,13.787249,3.974884,-37.084043,26.304949,14.705192,1.152176,...,11.722754,0.113290,10.052217,19.500012,-5.194602,29.852081,19.867939,27.586169,-17.435621,25.423339
Strategy Sharpe Ratio,-0.013176,0.157752,0.062819,0.074725,-0.003177,-0.025552,-0.098745,0.008537,-0.032985,0.004316,...,-0.012614,-0.080747,-0.029696,0.055179,-0.083869,0.062638,0.012766,0.001699,-0.058338,0.065152
Benchmark Sharpe Ratio,-0.098475,0.177281,0.102121,0.034533,0.189946,0.033586,-0.180453,0.088091,0.059559,0.005979,...,0.114446,0.000691,0.078450,0.384891,-0.028997,0.169809,0.058686,0.185516,-0.068954,0.129578


### Buying only if the current price is above the 6-month MA

In [240]:
# Convert the total_returns_dict to a DataFrame
total_returns_df_ma = pd.DataFrame.from_dict(above_ma_returns, orient='index', columns=['Total Returns'])
total_returns_df_ma.sort_index(inplace=True)


In [241]:
# Create a Plotly figure
fig = go.Figure()

# Add the benchmark returns bar trace
fig.add_trace(
    go.Bar(
        x=gains.index,
        y=gains.values,
        name='Benchmark Returns'
    )
)

# Add the momentum returns bar trace
fig.add_trace(
    go.Bar(
        x=total_returns_df_ma.index,
        y=total_returns_df_ma['Total Returns'],
        name='Strategy Returns'
    )
)

# Update the layout to include a range slider
fig.update_layout(
    title=' Price > 6-ma vs Benchmark Return from 01-01-2000',
    xaxis_title='Date',
    yaxis_title='Return %',
    barmode='group',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

# Show the plot
fig.show()

In [242]:
# Ensure both DataFrames have the same index for correct comparison
aligned_benchmark_gains_ma = gains.reindex(total_returns_df_ma.index).fillna(0)

# Calculate cumulative returns
cumulative_benchmark_returns_ma = (1 + aligned_benchmark_gains_ma / 100).cumprod() - 1
cumulative_momentum_returns_ma = (1 + total_returns_df_ma['Total Returns'] / 100).cumprod() - 1


In [243]:
# Create a Plotly figure
fig = go.Figure()

# Add the cumulative benchmark returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_benchmark_returns_ma.index,
        y=cumulative_benchmark_returns_ma,
        mode='lines',
        name='Cumulative Benchmark Returns'
    )
)

# Add the cumulative momentum returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_momentum_returns_ma.index,
        y=cumulative_momentum_returns_ma,
        mode='lines',
        name='Cumulative Strategy Returns'
    )
)

# Configure the x-axis to show months when zoomed in and add range selector buttons
fig.update_layout(
    title='Cumulative Returns: Price >6-ma vs Benchmark from 01-01-2001',
    xaxis_title='Date',
    yaxis_title='Cumulative Return %',
    xaxis=dict(
        tickmode='auto',
        rangeslider=dict(
            visible=True
        ),
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        type="date"
    ),
)

# Show the plot
fig.show()

In [245]:
# Calculate the yearly stats
yearly_stats_df = calculate_yearly_stats(above_ma_returns, gains)

yearly_stats_df.T

Year,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Strategy Returns,-15.052265,27.595732,12.122390,26.920382,-1.576293,5.588424,-36.185759,26.257141,19.828617,-3.123712,...,20.274201,3.829174,-12.910220,31.219004,-1.807606,14.166505,21.821744,-21.831322,-7.940742,17.493379
Benchmark Returns,-21.809476,27.099534,9.254411,3.286917,13.787249,3.974884,-37.084043,26.304949,14.705192,1.152176,...,11.722754,0.113290,10.052217,19.500012,-5.194602,29.852081,19.867939,27.586169,-17.435621,25.423339
Strategy Sharpe Ratio,-0.099431,0.164408,0.066574,0.123340,-0.007721,0.023151,-0.118449,0.094482,0.071058,-0.016186,...,0.049581,0.027162,-0.088268,0.071116,-0.005697,0.083637,0.085448,-0.065516,-0.036967,0.075041
Benchmark Sharpe Ratio,-0.098475,0.177281,0.102121,0.034533,0.189946,0.033586,-0.180453,0.088091,0.059559,0.005979,...,0.114446,0.000691,0.078450,0.384891,-0.028997,0.169809,0.058686,0.185516,-0.068954,0.129578


## Inversion Strategy

#### long the bottom 10%

In [235]:
# Convert the total_returns_dict to a DataFrame
total_returns_df_inversion = pd.DataFrame.from_dict(y, orient='index', columns=['Total Returns'])
total_returns_df_inversion.sort_index(inplace=True)


In [236]:
# Create a Plotly figure
fig = go.Figure()

# Add the benchmark returns bar trace
fig.add_trace(
    go.Bar(
        x=gains.index,
        y=gains.values,
        name='Benchmark Returns'
    )
)

# Add the momentum returns bar trace
fig.add_trace(
    go.Bar(
        x=total_returns_df_inversion.index,
        y=total_returns_df_inversion['Total Returns'],
        name='12-1 Momentum Returns'
    )
)

# Update the layout to include a range slider
fig.update_layout(
    title=' Inversion Return vs Benchmark Return from 01-01-2000',
    xaxis_title='Date',
    yaxis_title='Return %',
    barmode='group',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

# Show the plot
fig.show()

In [237]:
# Ensure both DataFrames have the same index for correct comparison
aligned_benchmark_gains = gains.reindex(total_returns_df_inversion.index).fillna(0)

# Calculate cumulative returns
cumulative_benchmark_returns_inversions = (1 + aligned_benchmark_gains / 100).cumprod() - 1
cumulative_momentum_returns_inversions = (1 + total_returns_df_inversion['Total Returns'] / 100).cumprod() - 1


In [238]:

# Create a Plotly figure
fig = go.Figure()

# Add the cumulative benchmark returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_benchmark_returns_ma.index,
        y=cumulative_benchmark_returns_ma,
        mode='lines',
        name='Cumulative Benchmark Returns'
    )
)

# Add the cumulative momentum returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_momentum_returns_inversions.index,
        y=cumulative_momentum_returns_inversions,
        mode='lines',
        name='12-1 Cumulative Momentum Returns'
    )
)

# Configure the x-axis to show months when zoomed in and add range selector buttons
fig.update_layout(
    title='Cumulative Returns: 12-1 Momentum vs Benchmark from 01-01-2001',
    xaxis_title='Date',
    yaxis_title='Cumulative Return %',
    xaxis=dict(
        tickmode='auto',
        rangeslider=dict(
            visible=True
        ),
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        type="date"
    ),
)

# Show the plot
fig.show()

In [259]:
# Calculate the yearly stats
yearly_stats_df = calculate_yearly_stats(long_dict_inversion, gains)

yearly_stats_df.T

Year,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Strategy Returns,-18.29,23.45,10.15,17.17,12.55,-4.14,-29.23,28.97,33.55,-22.44,...,-1.57,-18.21,0.37,20.45,5.09,7.27,47.59,10.05,-6.04,-0.74
Benchmark Returns,-21.81,27.10,9.25,3.29,13.79,3.97,-37.08,26.30,14.71,1.15,...,11.72,0.11,10.05,19.50,-5.19,29.85,19.87,27.59,-17.44,25.42
Strategy Sharpe Ratio,-0.07,0.12,0.10,0.10,0.14,-0.02,-0.11,0.09,0.15,-0.13,...,-0.07,-0.08,0.00,0.11,0.01,0.03,0.10,0.07,-0.02,-0.00
Benchmark Sharpe Ratio,-0.10,0.18,0.10,0.03,0.19,0.03,-0.18,0.09,0.06,0.01,...,0.11,0.00,0.08,0.38,-0.03,0.17,0.06,0.19,-0.07,0.13


#### Long on bottom 10% and short on top 10%

In [ ]:
# Convert the total_returns_dict to a DataFrame
s_l_total_returns_df_inversion = pd.DataFrame.from_dict(sort_long_dict_inversion, orient='index', columns=['Total Returns'])
s_l_total_returns_df_inversion.sort_index(inplace=True)


In [ ]:
# Create a Plotly figure
fig = go.Figure()

# Add the benchmark returns bar trace
fig.add_trace(
    go.Bar(
        x=gains.index,
        y=gains.values,
        name='Benchmark Returns'
    )
)

# Add the momentum returns bar trace
fig.add_trace(
    go.Bar(
        x=s_l_total_returns_df_inversion.index,
        y=s_l_total_returns_df_inversion['Total Returns'],
        name='12-1 Momentum Returns'
    )
)

# Update the layout to include a range slider
fig.update_layout(
    title=' Inversion Return(sort and long) vs Benchmark Return from 01-01-2000',
    xaxis_title='Date',
    yaxis_title='Return %',
    barmode='group',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

# Show the plot
fig.show()

In [ ]:
# Ensure both DataFrames have the same index for correct comparison
s_l_aligned_benchmark_gains = gains.reindex(s_l_total_returns_df_inversion.index).fillna(0)

# Calculate cumulative returns
s_l_cumulative_benchmark_returns_inversions = (1 + s_l_aligned_benchmark_gains / 100).cumprod() - 1
s_l_cumulative_momentum_returns_inversions = (1 + s_l_total_returns_df_inversion['Total Returns'] / 100).cumprod() - 1


In [ ]:

# Create a Plotly figure
fig = go.Figure()

# Add the cumulative benchmark returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_benchmark_returns_inversions.index,
        y=cumulative_benchmark_returns_inversions,
        mode='lines',
        name='Cumulative Benchmark Returns'
    )
)

# Add the cumulative momentum returns line trace
fig.add_trace(
    go.Scatter(
        x=s_l_cumulative_momentum_returns_inversions.index,
        y=s_l_cumulative_momentum_returns_inversions,
        mode='lines',
        name='Momentum inversion Returns'
    )
)

# Configure the x-axis to show months when zoomed in and add range selector buttons
fig.update_layout(
    title='Cumulative Returns: Inversion vs Benchmark from 01-01-2001',
    xaxis_title='Date',
    yaxis_title='Cumulative Return %',
    xaxis=dict(
        tickmode='auto',
        rangeslider=dict(
            visible=True
        ),
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        type="date"
    ),
)

# Show the plot
fig.show()

In [ ]:
# Calculate the yearly stats
yearly_stats_df = calculate_yearly_stats(sort_long_dict_inversion, gains)

yearly_stats_df.T

C:\Users\satya\AppData\Local\Temp\ipykernel_25332\3420669744.py:5: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.

C:\Users\satya\AppData\Local\Temp\ipykernel_25332\3420669744.py:17: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.



Year,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Strategy Returns,-9.613155,-8.778563,19.216087,4.995825,-8.615698,9.805042,-23.200976,-6.136139,37.104537,7.822143,...,-14.065914,14.457479,-19.106177,-1.846799,-5.161210,6.833471,21.271645,-11.482898,-12.564045,-5.102908
Benchmark Returns,-15.953767,-23.365964,26.380400,8.993452,3.001023,13.619432,3.529573,-38.485793,23.454190,12.782714,...,-0.726600,9.535023,19.419966,-6.237260,28.878070,16.258923,26.892739,-19.442827,24.230501,5.573781
Strategy Sharpe Ratio,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547,...,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547
Benchmark Sharpe Ratio,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,...,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469


## Seasonality

#### Buy from November to May

In [246]:
# Ensure both DataFrames have the same index for correct comparison
strategy_gains = trade_returns_seasonality.reindex(gains.index).fillna(0)

# Calculate cumulative returns
cumulative_benchmark_returns = (1 + gains / 100).cumprod() - 1
seasonality_cumulative_momentum_returns = (1 + strategy_gains['Trade Returns'] / 100).cumprod() - 1


In [65]:

# Create a Plotly figure
fig = go.Figure()

# Add the cumulative benchmark returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_benchmark_returns.index,
        y=cumulative_benchmark_returns,
        mode='lines',
        name='Cumulative Benchmark Returns'
    )
)

# Add the cumulative momentum returns line trace
fig.add_trace(
    go.Scatter(
        x=seasonality_cumulative_momentum_returns.index,
        y=seasonality_cumulative_momentum_returns,
        mode='lines',
        name='Seasonality Returns'
    )
)

# Configure the x-axis to show months when zoomed in and add range selector buttons
fig.update_layout(
    title='Cumulative Returns: Seasonality vs Benchmark from 01-01-2001',
    xaxis_title='Date',
    yaxis_title='Cumulative Return %',
    xaxis=dict(
        tickmode='auto',
        rangeslider=dict(
            visible=True
        ),
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        type="date"
    ),
)

# Show the plot
fig.show()

In [69]:
# Calculate the yearly stats
returns_dict = strategy_gains['Trade Returns'].to_dict()
yearly_stats_df = calculate_yearly_stats(returns_dict, gains)

yearly_stats_df.T

Year,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Strategy Returns,-0.651320,17.707882,5.426715,25.619926,6.431500,-1.451826,10.243641,3.226521,8.934173,14.422270,...,11.383630,-0.255834,13.090773,9.839417,4.011626,12.345803,6.217432,6.595306,2.771132,4.585997
Benchmark Returns,-21.809476,27.099534,9.254411,3.286917,13.787249,3.974884,-37.084043,26.304949,14.705192,1.152176,...,11.722754,0.113290,10.052217,19.500012,-5.194602,29.852081,19.867939,27.586169,-17.435621,25.423339
Strategy Sharpe Ratio,-0.012233,0.186803,0.058056,0.112544,0.047395,-0.009473,0.053902,0.013744,0.042535,0.167238,...,0.090661,-0.002506,0.130635,0.087911,0.052355,0.101552,0.025560,0.055783,0.011035,0.042467
Benchmark Sharpe Ratio,-0.098475,0.177281,0.102121,0.034533,0.189946,0.033586,-0.180453,0.088091,0.059559,0.005979,...,0.114446,0.000691,0.078450,0.384891,-0.028997,0.169809,0.058686,0.185516,-0.068954,0.129578


#### Buy only if stock increased > p% last year same month

In [407]:
# Convert the total_returns_dict to a DataFrame
total_returns_df = pd.DataFrame.from_dict(m, orient='index', columns=['Total Returns'])
total_returns_df.sort_index(inplace=True)


In [408]:
# Create a Plotly figure
fig = go.Figure()

# Add the benchmark returns bar trace
fig.add_trace(
    go.Bar(
        x=gains.index,
        y=gains.values,
        name='Benchmark Returns'
    )
)

# Add the momentum returns bar trace
fig.add_trace(
    go.Bar(
        x=total_returns_df.index,
        y=total_returns_df['Total Returns'],
        name='Momentum Returns'
    )
)

# Update the layout to include a range slider
fig.update_layout(
    title='Momentum Return vs Benchmark Return from 01-01-2000',
    xaxis_title='Date',
    yaxis_title='Return %',
    barmode='group',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

# Show the plot
fig.show()

In [409]:
# Ensure both DataFrames have the same index for correct comparison
aligned_benchmark_gains = gains.reindex(total_returns_df.index).fillna(0)

# Calculate cumulative returns
cumulative_benchmark_returns = (1 + aligned_benchmark_gains / 100).cumprod() - 1
cumulative_momentum_returns = (1 + total_returns_df['Total Returns'] / 100).cumprod() - 1


In [ ]:

# Create a Plotly figure
fig = go.Figure()

# Add the cumulative benchmark returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_benchmark_returns.index,
        y=cumulative_benchmark_returns,
        mode='lines',
        name='Cumulative Benchmark Returns'
    )
)

# Add the cumulative momentum returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_momentum_returns.index,
        y=cumulative_momentum_returns,
        mode='lines',
        name='Cumulative Momentum Returns'
    )
)

# Configure the x-axis to show months when zoomed in and add range selector buttons
fig.update_layout(
    title='Cumulative Returns: Momentum vs Benchmark from 01-01-2000',
    xaxis_title='Date',
    yaxis_title='Cumulative Return %',
    xaxis=dict(
        tickmode='auto',
        rangeslider=dict(
            visible=True
        ),
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        type="date"
    ),
)

# Show the plot
fig.show()

______________________________________________________________________________________________________________________________________________________

# NIFTY 500

In [262]:
url = 'https://en.wikipedia.org/wiki/NIFTY_500'

In [329]:
nse_symbols = list(pd.read_csv("C:/Users/satya/Downloads/ind_nifty500list.csv")['Company Name'])

In [301]:

nse_symbols1 = list(pd.read_html(url)[2][3])

In [339]:
symbols = []
for i in nse_symbols:
    x = i.split(' ')[0].upper()
    symbols.append(f'{x}.NS')

In [340]:
symbols

['360.NS',
 '3M.NS',
 'ABB.NS',
 'ACC.NS',
 'AIA.NS',
 'APL.NS',
 'AU.NS',
 'AARTI.NS',
 'AAVAS.NS',
 'ABBOTT.NS',
 'ACTION.NS',
 'ADANI.NS',
 'ADANI.NS',
 'ADANI.NS',
 'ADANI.NS',
 'ADANI.NS',
 'ADANI.NS',
 'ADANI.NS',
 'ADITYA.NS',
 'ADITYA.NS',
 'AEGIS.NS',
 'AETHER.NS',
 'AFFLE.NS',
 'AJANTA.NS',
 'ALEMBIC.NS',
 'ALKEM.NS',
 'ALKYL.NS',
 'ALLCARGO.NS',
 'ALOK.NS',
 'AMARA.NS',
 'AMBER.NS',
 'AMBUJA.NS',
 'ANAND.NS',
 'ANGEL.NS',
 'ANUPAM.NS',
 'APAR.NS',
 'APOLLO.NS',
 'APOLLO.NS',
 'APTUS.NS',
 'ARCHEAN.NS',
 'ASAHI.NS',
 'ASHOK.NS',
 'ASIAN.NS',
 'ASTER.NS',
 'ASTRAZENCA.NS',
 'ASTRAL.NS',
 'ATUL.NS',
 'AUROBINDO.NS',
 'AVANTI.NS',
 'AVENUE.NS',
 'AXIS.NS',
 'BEML.NS',
 'BLS.NS',
 'BSE.NS',
 'BAJAJ.NS',
 'BAJAJ.NS',
 'BAJAJ.NS',
 'BAJAJ.NS',
 'BALAJI.NS',
 'BALKRISHNA.NS',
 'BALRAMPUR.NS',
 'BANDHAN.NS',
 'BANK.NS',
 'BANK.NS',
 'BANK.NS',
 'BATA.NS',
 'BAYER.NS',
 'BERGER.NS',
 'BHARAT.NS',
 'BHARAT.NS',
 'BHARAT.NS',
 'BHARAT.NS',
 'BHARAT.NS',
 'BHARTI.NS',
 'BIKAJI.NS',
 'BIO

In [341]:
nse_stocks_data = {}

In [342]:
for i in symbols[:10]:
    nse_stocks_data[i] = yf.download(i, start_date, end_date)

[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['360.NS']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['3M.NS']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AIA.NS']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['APL.NS']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AU.NS']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[******************

In [343]:
nse_stocks_data

{'360.NS': Empty DataFrame
 Columns: [Open, High, Low, Close, Adj Close, Volume]
 Index: [],
 '3M.NS': Empty DataFrame
 Columns: [Open, High, Low, Close, Adj Close, Volume]
 Index: [],
 'ABB.NS':                    Open         High          Low        Close    Adj Close  \
 Date                                                                          
 2002-07-01     9.832109    10.186174     9.703193     9.875686     7.479980   
 2002-07-02    10.071784    10.073600     9.735876     9.821215     7.438723   
 2002-07-03     9.895659    10.168017     9.741323    10.089941     7.642259   
 2002-07-04    10.089941    10.309643    10.059074    10.148044     7.686268   
 2002-07-05    10.211594    10.454900    10.168017    10.247908     7.761905   
 ...                 ...          ...          ...          ...          ...   
 2023-12-22  4723.000000  4765.000000  4686.100098  4759.000000  4742.050781   
 2023-12-26  4771.350098  4824.000000  4747.250000  4771.350098  4754.356445   
 2023

________________________________________________________________________________________

# Extras

In [364]:
def calculate_monthly_percentage_change(stock_data):
    # Resample to get end-of-month data
    monthly_data = stock_data['Adj Close'].resample('ME').last()
    return monthly_data.pct_change() * 100

def classify_stocks(snp_stocks_data, p):
    long_dict = {}
    short_dict = {}
    
    for symbol, data in snp_stocks_data.items():
        # Calculate monthly percentage change
        monthly_pct_change = calculate_monthly_percentage_change(data)
        
        # Iterate through each month from the previous year
        for current_date in monthly_pct_change.index:
            if current_date.year >= 2001:  # Skip the first year since we need the previous year data
                last_year_date = current_date - pd.DateOffset(years=1)
                
                if last_year_date in monthly_pct_change.index:
                    pct_change = monthly_pct_change[last_year_date]
                    
                    # Classify into long or short based on percentage change
                    if pct_change >= p:
                        if current_date not in long_dict:
                            long_dict[current_date] = []
                        long_dict[current_date].append(symbol)
                    elif pct_change <= -p:
                        if current_date not in short_dict:
                            short_dict[current_date] = []
                        short_dict[current_date].append(symbol)
    
    return long_dict, short_dict

In [365]:
l, s = classify_stocks(snp_stocks_data, 10)

In [366]:
l

{Timestamp('2001-12-31 00:00:00'): ['MMM',
  'APD',
  'ALB',
  'ALL',
  'MO',
  'AMT',
  'AME',
  'ANSS',
  'AON',
  'APA',
  'ADM',
  'AZO',
  'BKR',
  'BALL',
  'BAC',
  'BBY',
  'BRO',
  'CDNS',
  'COF',
  'CCL',
  'CAT',
  'CNP',
  'CHRW',
  'CRL',
  'CMS',
  'CMCSA',
  'CMA',
  'STZ',
  'COO',
  'CPRT',
  'COST',
  'CTRA',
  'CCI',
  'DVA',
  'DECK',
  'DE',
  'DVN',
  'D',
  'DHI',
  'DD',
  'EMN',
  'EW',
  'EA',
  'EOG',
  'EQT',
  'EQR',
  'EG',
  'ES',
  'FDS',
  'FICO',
  'FITB',
  'GRMN',
  'GPC',
  'GS',
  'HSIC',
  'HES',
  'HD',
  'HST',
  'HUM',
  'IP',
  'IVZ',
  'IRM',
  'JBHT',
  'JKHY',
  'JPM',
  'KEY',
  'KLAC',
  'LH',
  'LEN',
  'LIN',
  'LOW',
  'MTB',
  'MAS',
  'MTCH',
  'MDT',
  'MET',
  'MTD',
  'MU',
  'MHK',
  'MS',
  'MOS',
  'NTAP',
  'NKE',
  'NI',
  'NUE',
  'NVR',
  'ORLY',
  'OXY',
  'OKE',
  'PH',
  'POOL',
  'PPG',
  'PGR',
  'PLD',
  'PEG',
  'PTC',
  'PSA',
  'DGX',
  'RJF',
  'RTX',
  'REGN',
  'RF',
  'RSG',
  'RMD',
  'RVTY',
  'ROK',
  'RCL'

In [367]:
m = total_return(snp_stocks_data, l, returns_df)

In [350]:
m

{Timestamp('2001-12-31 00:00:00'): -3.1799763886470838,
 Timestamp('2003-01-31 00:00:00'): -4.410973277479696,
 Timestamp('2003-04-30 00:00:00'): 18.893002603730856,
 Timestamp('2003-11-30 00:00:00'): 2.1716916747420565,
 Timestamp('2007-01-31 00:00:00'): -1.2231316037612951,
 Timestamp('2008-07-31 00:00:00'): -3.175570349642788,
 Timestamp('2009-07-31 00:00:00'): 0.4814500605089261,
 Timestamp('2010-04-30 00:00:00'): -0.47962190135166255,
 Timestamp('2010-07-31 00:00:00'): -9.366353012692398,
 Timestamp('2022-10-31 00:00:00'): 0.13999755529173688,
 Timestamp('2014-01-31 00:00:00'): 0.013136529381468804,
 Timestamp('2019-11-30 00:00:00'): 0.8295685161828875,
 Timestamp('2021-11-30 00:00:00'): -0.8154780759072142,
 Timestamp('2002-07-31 00:00:00'): -0.16467054785782104,
 Timestamp('2002-10-31 00:00:00'): 0,
 Timestamp('2002-11-30 00:00:00'): -10.888018864665442,
 Timestamp('2002-03-31 00:00:00'): -5.822863185420247,
 Timestamp('2002-04-30 00:00:00'): -5.116112049258549,
 Timestamp('2003

In [231]:
x = bottom_p_percentile_stocks(m, 33)

In [232]:
x

{Timestamp('2000-12-31 00:00:00'): ['BKNG',
  'FFIV',
  'XRX',
  'ILMN',
  'VRSN',
  'NVDA',
  'AKAM',
  'AMZN',
  'WDC',
  'JBL',
  'BBY',
  'EBAY',
  'ON',
  'EQIX',
  'NTAP',
  'GLW',
  'URI',
  'AMD',
  'JNPR',
  'AAPL',
  'IT',
  'DLTR',
  'INCY',
  'OI',
  'ADI',
  'CSGP',
  'SLR',
  'AMAT',
  'TECH',
  'TWX',
  'MOLX',
  'HPQ',
  'CIEN',
  'MCHP',
  'PENN',
  'SEE',
  'APH',
  'INTU',
  'CSCO',
  'LRCX',
  'MSI',
  'ABMD',
  'MNST',
  'HOLX',
  'CFC',
  'MSFT',
  'ISRG',
  'INTC',
  'ORCL',
  'MGM',
  'HAL',
  'ADBE',
  'IBM',
  'GILD',
  'GEN',
  'DIS',
  'RHI',
  'URBN',
  'MU',
  'TSCO',
  'BBWI',
  'RIG',
  'BIG',
  'FI',
  'SCHW',
  'CRL',
  'AES',
  'DXC',
  'TDY',
  'EW',
  'KLAC',
  'EIX',
  'IDXX',
  'TIE',
  'HOG',
  'GE',
  'BSX',
  'TRB',
  'PCG',
  'TYL',
  'VRTX',
  'PBI',
  'ZBRA',
  'BEN',
  'KMX',
  'EA',
  'HD',
  'DOV',
  'MS',
  'CCI',
  'AON',
  'ATI',
  'MMC',
  'MTCH',
  'BAC',
  'ABK',
  'NKTR',
  'FDX',
  'NDSN',
  'CLX',
  'TROW',
  'AXP',
  'BC',
  'R'

In [233]:
y = total_return(snp_stocks_data, x, returns_df)

In [234]:
y

{Timestamp('2000-12-31 00:00:00'): -0.12037278265855186,
 Timestamp('2001-01-31 00:00:00'): -5.6790574398382425,
 Timestamp('2001-02-28 00:00:00'): -3.6711382036499605,
 Timestamp('2001-03-31 00:00:00'): 5.243621060689314,
 Timestamp('2001-04-30 00:00:00'): -1.0932516571895168,
 Timestamp('2001-05-31 00:00:00'): -4.4831258520023365,
 Timestamp('2001-06-30 00:00:00'): -3.4536759420951877,
 Timestamp('2001-07-31 00:00:00'): -7.72981086421337,
 Timestamp('2001-08-31 00:00:00'): -5.064271364529689,
 Timestamp('2001-09-30 00:00:00'): 1.5279231334866545,
 Timestamp('2001-10-31 00:00:00'): 8.599486360818286,
 Timestamp('2001-11-30 00:00:00'): 4.778241583773404,
 Timestamp('2001-12-31 00:00:00'): -0.681202616047016,
 Timestamp('2002-01-31 00:00:00'): 1.6868647376385997,
 Timestamp('2002-02-28 00:00:00'): 4.958306082094267,
 Timestamp('2002-03-31 00:00:00'): -3.8608780593864958,
 Timestamp('2002-04-30 00:00:00'): -1.8311706056222847,
 Timestamp('2002-05-31 00:00:00'): -3.5579033663085555,
 Time

In [ ]:
mont_r.shift(-1)

Date
2000-01-31    1366.420044
2000-02-29    1498.579956
2000-03-31    1452.430054
2000-04-30    1420.599976
2000-05-31    1454.599976
                 ...     
2024-02-29    5254.350098
2024-03-31    5035.689941
2024-04-30    5277.509766
2024-05-31    5283.399902
2024-06-30            NaN
Freq: ME, Name: Adj Close, Length: 294, dtype: float64